In [1]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
# using Interpolations

In [2]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

get_control_int (generic function with 1 method)

In [ ]:
# --- Problemparametere ---
h = 1.0
k = 1.0
ρ = 1.0
c = 1.0
θ = 0.5
Δt = 0.05
t0 = 0.0
tF = 1.0
h_wall = 1.0

# --- Manufactured solution og avledede størrelser ---
T_exact(x, t) = sin(x[1] + x[2] + t)
T_t(x, t) = cos(x[1] + x[2] + t)
ΔT(x, t) = -2 * sin(x[1] + x[2] + t)
Q(x, t) = T_t(x, t) - ΔT(x, t)

T_out_up(x, t) = cos(x[1] + x[2] + t) - T_exact(x, t)
T_out_down(x, t) = -cos(x[1] + x[2] + t) - T_exact(x, t)
T_out_left(x, t) = -cos(x[1] + x[2] + t) - T_exact(x, t)
T_out_right(x, t) = cos(x[1] + x[2] + t) - T_exact(x, t)

Qt(t) = x -> Q(x, t)

# --- Initialtilstand ---
TIni(x) = T_exact(x, t0)

# --- Geometri og mesh ---
domain = (0.0, 1.0, 0.0, 1.0)
partition = (25, 25)
model = CartesianDiscreteModel(domain, partition)

labels = get_face_labeling(model)
add_tag_from_tags!(labels, "left", [1,3,7])
add_tag_from_tags!(labels, "right", [2,4,8])
add_tag_from_tags!(labels, "bottom", [5])
add_tag_from_tags!(labels, "top", [6])

Ω = Triangulation(model)
Γ_up = BoundaryTriangulation(model, tags="top")
Γ_down = BoundaryTriangulation(model, tags="bottom")
Γ_left = BoundaryTriangulation(model, tags="left")
Γ_right = BoundaryTriangulation(model, tags="right")

order = 1
degree = 2 * order
dΩ = Measure(Ω, degree)
dΓ_up = Measure(Γ_up, degree)
dΓ_down = Measure(Γ_down, degree)
dΓ_left = Measure(Γ_left, degree)
dΓ_right = Measure(Γ_right, degree)

# --- Rom og FE-definisjon ---
reffe = ReferenceFE(lagrangian, Float64, order)

Testspace = TestFESpace(model, reffe; conformity=:H1)
Trialspace = TransientTrialFESpace(Testspace)

# --- Svak form ---
a_dt(t, dT, v) = ∫(c * dT * v * ρ) * dΩ

a_SE(t, T, v) = ∫(k * ∇(T) ⋅ ∇(v)) * dΩ +
                ∫(h_wall * T * v) * dΓ_up +
                ∫(h_wall * T * v) * dΓ_down +
                ∫(h_wall * T * v) * dΓ_left +
                ∫(h_wall * T * v) * dΓ_right

l_SE(t, v) = ∫(Qt(t) * v) * dΩ +
             ∫(x -> T_out_up(x, t) * h_wall * v(x)) * dΓ_up +
             ∫(x -> T_out_down(x, t) * h_wall * v(x)) * dΓ_down +
             ∫(x -> T_out_left(x, t) * h_wall * v(x)) * dΓ_left +
             ∫(x -> T_out_right(x, t) * h_wall * v(x)) * dΓ_right

# --- Operator og løsning ---
ls = LUSolver()
solver = ThetaMethod(ls, Δt, θ)

op = TransientLinearFEOperator((a_dt, a_SE), l_SE, Trialspace, Testspace, constant_forms=(true, true))

T0 = interpolate(TIni, Trialspace)
T = solve(solver, op, t0, tF, T0)

GenericTransientFESolution()

In [ ]:
# Definer tidsstegene og løsningen
T = [(t0, T0)]  # Start med initialbetingelsen
for (t, TT) in zip(timesteps, computed_solutions)
    push!(T, (t, FEFunction(Trialspace, copy(get_free_dof_values(TT)))))
end

# Lagre løsningene som VTK-filer og lag PVD-filen
if !isdir("tmp_M")
    mkdir("tmp_M")
end

createpvd("results_M") do pvd
    # Lagre den initiale løsningen ved t = t0
    pvd[0] = createvtk(Ω, "tmp_M/results_0.vtu", cellfields=["T" => T0])
    
    # Lagre løsningene for hvert tidssteg
    for (tn, (t, TT)) in enumerate(T)
        # Hent FE-funksjonen for løsningen ved tidssteg t
        Tn = FEFunction(Trialspace, copy(get_free_dof_values(TT)))
        
        # Lagre løsningen som en VTK-fil
        pvd[tn] = createvtk(Ω, "tmp_M/results_$tn.vtu", cellfields=["T" => Tn])
    end
end



UndefVarError: UndefVarError: `timesteps` not defined in `Main`
Suggestion: check for spelling errors or missing imports.